# Tic-Tac-Toe (Events + Observers)

This notebook uses event-style inputs with a `events` namespace.
Human plays X, AI plays O. The widget emits outputs, and Python responds by issuing events.

In [ ]:
import time
import vibe_widget as vw

## Create the game widget

In [ ]:
game_board = vw.create(
    """Interactive Tic-Tac-Toe game board
    - Human plays X, AI plays O
    - Click cells to make moves
    - Outputs board_state, current_turn, game_over
    - Listen for custom events: ai_move(index: 0-8, row-major)
    - Handle events via model.on('msg:custom', ...) and apply ai_move
    """,
    outputs=vw.outputs(
        board_state="9-element array of 'x', 'o', or 'b'",
        game_over="boolean",
        current_turn="'x' or 'o'"
    ),
    actions=vw.actions(
        ai_move=vw.action(
            "index 0-8 (row-major) for the AI's move",
            params={"index": "0-8 row-major"}
        )
    ),
    # cache=False
)

game_board

## AI move logic

In [ ]:
def pick_first_empty(board_list):
    for idx, cell in enumerate(board_list):
        if cell == "b":
            return idx
    return None

## Observe outputs and issue events

In [ ]:
game_board = vw.create(
    """Interactive Tic-Tac-Toe game board
    - Human plays X, AI plays O
    - Click cells to make moves
    - Outputs board_state, current_turn, game_over
    - Listen for custom events: ai_move(index: 0-8, row-major)
    - Handle events via model.on('msg:custom', ...) and apply ai_move
    - make sure to use a functional state update or a ref so the AI move applies on the latest board.
    """,
    outputs=vw.outputs(
        board_state="9-element array of 'x', 'o', or 'b'",
        game_over="boolean",
        current_turn="'x' or 'o'"
    ),
    actions=vw.actions(
        ai_move=vw.action(
            "index 0-8 (row-major) for the AI's move",
            params={"index": "0-8 row-major"}
        )
    ),
    cache=False
)

game_board


def pick_first_empty(board_list): 
    for idx, cell in enumerate(board_list):
        if cell == "b":
            return idx
    return None

def on_turn_change(event):
    if event.new != "o":
        return

    # Allow the frontend to finish updating its state.
    time.sleep(0.1)

    board_state = game_board.outputs.board_state.value
    if not board_state or game_board.outputs.game_over.value:
        return

    if isinstance(board_state, str):
        import ast
        board_state = ast.literal_eval(board_state)

    board_list = list(board_state)
    if len(board_list) != 9:
        return

    move_index = pick_first_empty(board_list) # can be replaced with a neural net
    if move_index is None:
        return

    game_board.actions.ai_move(index=move_index)


game_board.outputs.current_turn.observe(on_turn_change)


In [ ]:
game_board.code